In [1]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
#Load csv files 
HourlyMCP = pd.read_csv('../Data/ModelData/HourlyMCPDemo.csv', delimiter=',')
Fuel_Cost_Forecast_Normalized = pd.read_csv('../Data/ModelData/FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Asset_Data_New = pd.read_csv('../Data/ModelData/Generators_AssetData_New.csv', delimiter=',')

In [3]:
#show the first 5 rows of the data
HourlyMCP.head()
Fuel_Cost_Forecast_Normalized.head()
Generation_Asset_Data_New.head()

,Unit #,Zone,Technology,P_max (MW),P_min (MW),P_max current (MW)
0,1,0,Coal,152,30.40,0
1,2,0,Coal,152,30.40,0
2,3,0,Coal,350,75.00,0
3,4,0,Gas,591,206.85,0
4,5,0,Gas,60,12.00,0
